## **Planner**

![skpatternlarge](./imgs/01/skpatternlarge.png)

Planner 是一个方法，是最终目标，结合 LLM 划分如何完成请求的计划。 它通过使用人工智能来混合和匹配在 Kernel 中注册的插件来实现，可以将它们重新组合来完成既定目标。


![skpatternlarge](./imgs/03/the-planner.png)

Planner 是 Semantic  Kernel 的重要部分。 这意味着我们有多个 Planner 可供选择，如果您有特定需求，您可以创建自定义 Planner。

- BasicPlanner SequentialPlanner 的简化版本，将一组函数串在一起。 
- ActionPlanner 只需一步即可创建计划。 
- SequentialPlanner 创建一个包含一系列步骤的计划，这些步骤与自定义生成的输入和输出变量互连。 
- StepwisePlanner 在执行下一步之前增量执行步骤并观察任何结果。

In [1]:
import os
import sys

import semantic_kernel as sk

In [2]:

kernel = sk.Kernel()

In [3]:
import semantic_kernel.connectors.ai.open_ai as skaoai

In [4]:
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
# kernel.add_text_completion_service("chat-completion-service", skaoai.AzureChatCompletion("GPT3", endpoint, api_key))
kernel.add_chat_service("azure_chat_competion_service", skaoai.AzureChatCompletion("GPT3TurboModel",endpoint,api_key=api_key,api_version = "2023-07-01-preview"))

In [5]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [6]:
base_skills_directory = './plugins'

In [7]:
custom_plugin = kernel.import_native_skill_from_directory(base_skills_directory , "APIPlugin")
writer_plugin = kernel.import_semantic_skill_from_directory(base_skills_directory, "WriterPlugin")
email_plugin = kernel.import_semantic_skill_from_directory(base_skills_directory, "EmailPlugin")
translate_plugin = kernel.import_semantic_skill_from_directory(base_skills_directory, "TransaltePlugin")

In [8]:
ask = """
Check the weather in Guangzhou and generate dressing tips based on the results and write them as travel weather reminder emails ，translate the results into spanish
"""
original_plan = await planner.create_plan_async(ask, kernel)

In [9]:
print(original_plan.generated_plan)

{
    "input": "Guangzhou Weather and Dressing Tips",
    "subtasks": [
        {"function": "APIPlugin.WeatherFunction", "args": {"city": "Guangzhou"}},
        {"function": "WriterPlugin.Tips", "args": {"language": "Spanish"}},
        {"function": "EmailPlugin.WeatherMail"}
    ]
}


In [10]:
await planner.execute_plan_async(original_plan, kernel)

'Hola,\n\nSolo quería recordarte que el clima va a estar bastante caluroso durante nuestro viaje, así que asegúrate de vestir ropa ligera y transpirable para mantenerte fresco. No te olvides de llevar contigo unas gafas de sol y un sombrero para protegerte del sol.\n\nEs importante que estemos bien hidratados, así que recuerda beber mucha agua durante el día.\n\nAdemás, evita la exposición directa al sol durante largos periodos de tiempo. Intenta buscar sombra cuando sea posible.\n\nPor favor, mantente atento a tu estado durante el viaje y no dudes en avisar si necesitas hacer alguna parada para descansar o refrescarte.\n\n¡Espero que tengamos un viaje seguro y agradable!\n\nSaludos, [Tu nombre]'